In [ ]:
!pip install tensorflow

In [ ]:
pip install keras-tuner

In [ ]:
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.optimizers import Adam
from keras_tuner import HyperModel, RandomSearch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [ ]:
data = pd.read_excel("NaturalGasDemand_Input.xlsx")
data

In [ ]:
X = data.drop(columns=["India total Consumption of Natural Gas (in BCM)","Month"]) # independent variables
y = data["India total Consumption of Natural Gas (in BCM)"]  # dependent variable
dates=data["Month"]


In [ ]:
X_train, X_test, y_train, y_test, dates_train, dates_test = train_test_split(X, y, dates, test_size=0.2,random_state=42)

In [ ]:
scaler = MinMaxScaler(feature_range=(-1, 1))
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
# Step 6: Build the neural network model
# Function to build a model (used for hyperparameter tuning)
def build_model(hp):
    model = Sequential()
    # Add input layer
    model.add(Dense(hp.Int('units_input', min_value=32, max_value=512, step=32),
                    activation='elu', input_dim=X_train_scaled.shape[1]))
    # Add hidden layers (tunable number of layers)
    for i in range(hp.Int('num_layers', 1, 50)):
        model.add(Dense(hp.Int(f'units_{i}', min_value=32, max_value=512, step=32),
                        activation='elu'))
    
    model.add(Dense(1))
    # Compile the model
    model.compile(optimizer='adam', loss='mean_absolute_error')
    return model

In [ ]:
# Step 7: Hyperparameter tuning using Keras Tuner
tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=10,
    executions_per_trial=10,
    directory='NaturalGasDemand_DNN_Fundamental_June2025_50ELU1',
    project_name='Proj_NaturalGasDemand_DNN_Fundamental_June2025_50ELU1')

In [ ]:
tuner.search(X_train, y_train, epochs=100, validation_split=0.1)

In [ ]:
best_hps = tuner.get_best_hyperparameters()[0]

In [ ]:
tuner.search_space_summary()

In [ ]:
# Build the model with the optimal hyperparameters
model = tuner.hypermodel.build(best_hps)


In [ ]:
# Train the model
model.fit(X_train_scaled, y_train, epochs=100, validation_split=0.1)


In [ ]:
model.summary()

In [ ]:
# Evaluate the model on the test set
test_loss = model.evaluate(X_test_scaled, y_test)
print(f'Test loss: {test_loss}')

In [ ]:
y_pred_train = model.predict(X_train_scaled)
y_pred_test = model.predict(X_test_scaled)

from sklearn.metrics import mean_squared_error, mean_absolute_error
# Calculate mean squared error
mae_train = mean_absolute_error(y_train, y_pred_train)
mae_test = mean_absolute_error(y_test, y_pred_test)
print(f'MAE on Training Data: {mae_train}')
print(f'MAE on Test Data: {mae_test}')

In [ ]:
# Plotting y_test vs y_pred
import matplotlib.pyplot as plt

plt.figure(figsize=(8, 6))
plt.plot(range(len(y_train)), y_train, color='black', alpha=0.5, label='Data Points')
plt.plot(range(len(y_pred_train)), y_pred_train, color='red', alpha=0.5, label='Data Points')
#plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=2, label='Ideal Fit')  # Diagonal line
plt.xlabel('Actual')
plt.ylabel('Predicted')
plt.title('Actual vs Predicted ICE Raw Sugar Prices')
plt.legend()
plt.show()

In [ ]:
# Predict using the model
predictions = model.predict(X_test_scaled)
print(predictions)
print(y_test)

In [ ]:
import matplotlib.pyplot as plt

# Plotting y_test vs y_pred
plt.figure(figsize=(8, 6))
plt.plot(range(len(y_test)), y_test, color='black', alpha=0.5, label='Actual')
plt.plot(range(len(predictions)), predictions, color='red', alpha=0.5, label='Predictions')
plt.xlabel('Range')
plt.ylabel('Price')
plt.title('Actual vs Predicted ICE Raw Sugar Prices')
plt.legend()
plt.show()

In [ ]:
train_results = pd.DataFrame({'Date': dates_train, 'Actual': y_train, 'Predicted': y_pred_train.flatten()})
test_results = pd.DataFrame({'Date': dates_test, 'Actual': y_test, 'Predicted': y_pred_test.flatten()})

train_results.to_excel('June25_NaturalGasDemand_ELU50_train_predictions.xlsx', index=False)
test_results.to_excel('June25_NaturalGasDemand_ELU50_test_predictions.xlsx', index=False)

In [ ]:
future_data = pd.read_excel("NaturalGasDemand_Future_Input.xlsx") 


In [ ]:
# Normalize the future data using the same scaler fitted on X_train
X_future = scaler.transform(future_data)

In [ ]:
# Preprocess future data

X_future

In [ ]:
# Make predictions
future_predictions = model.predict(X_future)


In [ ]:
#future_predictions

In [ ]:
future_predictions = pd.DataFrame(future_predictions)

In [ ]:
#future_predictions.to_excel("NaturalGasDemand_ELU50_Forecasts.xlsx")